<a href="https://colab.research.google.com/github/msjun23/Deep-Learning-from-Scratch/blob/main/Chapter6/1_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최적화

손실 함수의 값을 가능한 낮추는 것.

**SGD**, 확률적 경사 하강법의 수식은 다음과 같이 쓸 수 있다.
$$\textbf W\leftarrow\textbf W-\eta\frac{\partial L}{\partial \textbf W}$$

여기서 $\eta$는 학습률(learning rate)이다.

In [1]:
# 확률적 경사 하강법(SGD)
class SGD:
  def __init__(self, lr=0.01):
    self.lr = lr

  def update(sefl, params, grads):
    for key in params.keys():
      params[key] -= self.lr * grads[key]

```
# pseudo code 예시
network = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
  ...
  x_batch, t_batch = get_mini_batch(...)
  grads = network.gradient(x_batch, t_batch)
  params = network.params
  optimizer.update(params, grads)
  ...
```


SGD의 단점

- 한 방향으로의 기울기가 가파르고 다른 방향으로의 기울기가 완만하다면, 기울기가 가파른 방향으로 요동치며 최적화가 진행된다. 이는 상당히 비효율적인 움직임이다.
- 비등방성(anisotropy) 함수에서는 탐색 경로가 비효율적이다.

---

**모멘텀**(Momentum)은 속도(가속)의 정보를 수식에 추가했다고 보면 된다.
$$\textbf v\leftarrow\alpha\textbf v-\eta\frac{\partial L}{\partial \textbf W}
\\
\textbf W\leftarrow\textbf W+\textbf v$$

공이 곡면(기울기)을 따라 움직이는 모습을 상상하면 된다. 움직일 수록 가속도를 받아 더욱 빠르게 경사를 따라 하강할 것이다. 위의 식에서 $\alpha\textbf v$항은 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할을 한다. $\alpha$는 보통 0.9 등으로 설정한다. 

In [2]:
# Momentum 구현
class Momentum:
  def __init__(self, lr=0.01, momentum=0.9):
    sefl.lr = lr
    self.momentum = momentum
    sefl.v = None

  def update(sefl, params, grads):
    if self.v is None:
      sefl.v = {}
      for key, val in params.items():
        self.v[key] = np.zeros_like(val)

    for key in params.keys():
      self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
      params[key] += self.v[key]

SGD와 비교했을 때 덜 지그재그로 움직인다.

---

**AdaGrad**는 각각의 매개변수에 적응적으로 학습률을 조정하면서 학습을 진행한다. AdaGrad의 수식은 다음과 같다.
$$\textbf h\leftarrow\textbf h+\frac{\partial L}{\partial\textbf W}\bigodot\frac{\partial L}{\partial\textbf W}
\\
\textbf W\leftarrow\textbf W-\eta\frac{1}{\sqrt{\textbf h}}\frac{\partial L}{\partial\textbf W}$$

AdaGrad는 기존 기울기의 값을 제곱하여 계속 더해준다($\textbf h$).그리고 매개변수를 갱신할 때 $\textbf h$의 제곱근으로 learning rate를 나눠 학습률을 조정한다.

매개변수의 원소 중에서 많이 움직인(크게 갱신되) 원소는 학습률이 낮아진다는 것이데, 다시 말해 학습률 감소가 매개변수의 원소마다 다르게 적용된다는 것이다.

In [3]:
# AdaGrad 구현
class AdaGrad:
  def __init__(self, lr=0.01):
    self.lr = lr
    self.h = None

  def update(self, params, grads):
    if self.h is None:
      self.h = {}
      for key, val in params.items():
        self.h[key] = np.zeros_like(val)
      
    for key in params.keys():
      self.h[key] += grads[key] * grads[key]
      params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

최솟값을 향해 효율적으로 움직인다. 예를 들어 y축 방향으로 기울기가 크다면, 처음에는 y축으로 크게 움직이지만, 그 큰 움직임에 비례해 갱신 정도도 큰 폭으로 작아지도록 조정된다.

---

모멘텀과 AdaGrad를 융합한 것이 **Adam**이다. 모멘텀과 비슷한 최적화 패턴을 보이나 모멘텀보다 반동이 적다.이는 학습의 갱신 정도를 적응적으로 조정해서 얻는 효과이다.